In [2]:
%matplotlib inline
import os
try:
    import Image
except ImportError:
    from PIL import Image
import cv2
import numpy as np
import pytesseract
from matplotlib import pyplot as plt
import math
from math import radians, degrees
from skewManager import imgproc

import pandas as pd
import json
from BaiduAipOcr import BaiduAipOcr,OcrApiPool

In [3]:


""" 你的 APPID AK SK """
APP_ID = '11507399'
API_KEY = 'DH6shd00MjvNolrX3WskgNmC'
SECRET_KEY = 'uO0FQ0P4c1zFQh0QvWBgeuFMfaKsRpq4'

client = BaiduAipOcr()

def get_file_content(filePath):
    with open(filePath, 'rb') as fp:
        return fp.read()
    
def image_to_stringAccurate(image):
    options = {}
    options["language_type"] = "CHN_ENG"
    text = ''
    result = client.basicAccurateforCv2(image, options)
    #print(result)
    words_result = result['words_result']
    line_num = result['words_result_num']
    for i in range(line_num):
        text += words_result[i]['words']
    return text

def image_to_stringGeneral(image):
    options = {}
    options["language_type"] = "CHN_ENG"
    text = ''
    result = client.basicGeneralforCv2(image, options)
    print(result)
    words_result = result['words_result']
    line_num = result['words_result_num']
    #for i in range(line_num):
    #    text += words_result[i]['words']
    return words_result
def imgtostring(filepath):


    options = {}
    options["language_type"] = "CHN_ENG"

    image = get_file_content(filepath)
    """ 带参数调用通用文字识别（含位置信息版）, 图片参数为本地图片 """
    resutl=client.basicAccurateforCv2(image, options)
    words_result = result['words_result']
    line_num = result['words_result_num']
    for i in range(line_num):
        text += words_result[i]['words']
    return text
def imgtostring2(filepath):


    options = {}
    options["language_type"] = "CHN_ENG"

    image = get_file_content(filepath)
    """ 带参数调用通用文字识别（含位置信息版）, 图片参数为本地图片 """
    resutl=client.basicGeneralforCv2(image, options)
    words_result = result['words_result']
    line_num = result['words_result_num']
    for i in range(line_num):
        text += words_result[i]['words']
    return text    
def imgtostring3(filepath):


    options = {}
    options["language_type"] = "CHN_ENG"

    image = get_file_content(filepath)
    """ 带参数调用通用文字识别（含位置信息版）, 图片参数为本地图片 """
    resutl=client.image_to_stringforfile(image, options)

    return resutl  

In [4]:
fpathe = "I:/陈宁/对账单 转格式 part 2/10/10 中国银行 0833_页面_001.jpg"
image = get_file_content(fpathe)

options = {}
options["vertexes_location"] = "true"
resutl =client.tableRecognitionAsync(image);
#resutl =client.accurate(image);
print(resutl)

{'result': [{'request_id': '14266691_654168'}], 'log_id': 15409915906159706}


In [10]:
""" 如果有可选参数 """
requestId = resutl['result'][0]['request_id']
options = {}
options["result_type"] = "excel"

""" 带参数调用表格识别结果 """

client.getTableRecognitionResult(requestId, options)

{'result': {'result_data': 'http://bj.bcebos.com/v1/aip-web/form_ocr/59B8112C676F474D90766F1AD73171BD.xls?authorization=bce-auth-v1%2Ff86a2044998643b5abc89b59158bad6d%2F2018-10-30T15%3A58%3A50Z%2F86400%2F%2Fcabd6c9d4ef665ccb408890858d46e240b6ced287cc6b570b58f3b15c860777f',
  'ret_msg': '已完成',
  'request_id': '14266691_650821',
  'percent': 100,
  'ret_code': 3},
 'log_id': 15409151365158710}

In [ ]:
import os

fpathe = './testimage'
print("测试接口 {} jpg".format(fpathe))
apipool = OcrApiPool(debug=True)
for fpath,dirs,fs in os.walk(fpathe):
    
    for file in fs:
        fname,extension = os.path.splitext(file)
        
        if extension != '.jpg':
            continue
        print("---- {} is start---".format(file))
        filepath = os.path.join(fpathe,file)
        #img = cv2.imread(filepath)
        iproc_obj = imgproc.ImageProc(filepath)
        imgbilateral= cv2.bilateralFilter(iproc_obj.gray_img, 11, 21, 11);   # 双边滤波，去除噪音，边缘不变
        meansThresh = cv2.fastNlMeansDenoising(imgbilateral, None, 27, 11, 7) # 中值滤波，去除噪音
        #ret1,th1=cv2.threshold(imgbilateral,200,255,cv2.THRESH_TRUNC)
        #clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(15,15))
        #cl1_img = clahe.apply(th1)  #直方图均衡
        #bilateral_img = cv2.bilateralFilter(cl1_img,3,21,21,cv2.BORDER_WRAP)
        sigma = 1; threshold = 2; amount = 1
        blurred=cv2.GaussianBlur(imgbilateral,(0,0),1,None,1)

        lowContrastMask = abs(imgbilateral - blurred) < threshold
        sharpened = blurred*(1+amount) + blurred*(-amount)
        tableImgClear=cv2.bitwise_or(sharpened.astype(np.uint8),lowContrastMask.astype(np.uint8))
        #thresh_img = cv2.adaptiveThreshold(iproc_obj.gray_img,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,35,15)
        ocrapi = apipool.get_ocrapi()
        #text1 = pytesseract.image_to_string(thresh_img, lang='chi_sim',boxes=False,config="--psm 3 --oem 1")
        text1 = image_to_stringGeneral(meansThresh)
        text2 = image_to_stringGeneral(iproc_obj.gray_img)
        text3 = image_to_stringGeneral(tableImgClear)
        #text = ocrapi.image_to_string(thresh_img)
        #apipool.add_ocrapi(ocrapi)
        print(text1)
        print(text2)
        print(text3)
        plt.ion()
        plt.figure(fname,figsize=(10, 40))
        plt.subplot(1,3,1),plt.imshow(meansThresh,'gray')
        plt.subplot(1,3,2),plt.imshow(iproc_obj.gray_img,'gray')
        
        plt.subplot(1,3,3),plt.imshow(tableImgClear,'gray')
        plt.pause(1)
        plt.close()



In [ ]:
import os

fpathe = './test3'
print("测试接口 {} jpg".format(fpathe))
apipool = OcrApiPool(debug=True)
for fpath,dirs,fs in os.walk(fpathe):
    
    for file in fs:
        fname,extension = os.path.splitext(file)
        
        if extension != '.jpg':
            continue
        print("---- {} is start---".format(file))
        filepath = os.path.join(fpathe,file)
        #img = cv2.imread(filepath)
        iproc_obj = imgproc.ImageProc(filepath)
        #thresh_img = cv2.adaptiveThreshold(
        #    ~iproc_obj.gray_img, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 9, -2)
        #text = pytesseract.image_to_string(thresh_img, 'eng+chi_sim')
        
        
        clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(15,15))
        cl1_img = clahe.apply(iproc_obj.gray_img)
        bilateral_img = cv2.bilateralFilter(cl1_img,3,21,21,cv2.BORDER_WRAP)

        #canny_img = cv2.Canny(bilateral_img, 100, 200)
        erode_structure = cv2.getStructuringElement(
            cv2.MORPH_CROSS, (3, 3))# 形态学因子
        
        
        imgThresh = cv2.threshold(~bilateral_img, 50, 255, cv2.THRESH_TOZERO)[1]
        #andimg = cv2.bitwise_and(thresh_img, dilate_img)
        
        addimg = cv2.add(~bilateral_img,~bilateral_img)
        thresh_img = cv2.adaptiveThreshold(~addimg,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,55,100)
        dilate_img = cv2.erode(thresh_img, erode_structure, 1)
        erode_imgT = cv2.dilate(dilate_img, erode_structure, 1)
        
        ocrapi = apipool.get_ocrapi()
        text1 = pytesseract.image_to_string(bilateral_img, lang='chi_sim',boxes=False,config="--psm 3 --oem 1")
        #text = image_to_stringGeneral(img)
        #text2 = ocrapi.image_to_string(~addimg)
        #text3= ocrapi.image_to_string(~imgThresh)
        #text4 = ocrapi.image_to_string(bilateral_img)
        #text5 = ocrapi.image_to_string(erode_imgT)
        apipool.add_ocrapi(ocrapi)
        #print(text1,'----',text2 ,'----',text3,'---',text4,'---',text5)
        print(text1)
        
        
        plt.ion()
        plt.figure(fname,figsize=(16, 4))
        plt.subplot(1,4,1),plt.imshow(~addimg,'gray')
        plt.subplot(1,4,2),plt.imshow(~imgThresh,'gray')
        plt.subplot(1,4,3),plt.imshow(bilateral_img,'gray')
        plt.subplot(1,4,4),plt.imshow(erode_imgT,'gray')
        plt.pause(1)
        plt.close()



In [ ]:
%load_ext autoreload
%autoreload 2
class Parent(object):
    x = 1
    __a =3 # 类变量
    def __init__(self):
        self.y = 1  # 实例变量
class Children1(Parent):
    def tet(self):
        print(self.x)
        print(self._Parent__a)
class Children2(Parent):
    pass
par = Parent()  # 具体的类实例对象
child1 = Children1()  # 具体的类实例对象
child2 = Children2()  # 具体的类实例对象
print('-------------------------------------')
print('Parent.x=', Parent.x, ',Children1.x=', Children1.x, ',Children2.x=', Children2.x)
print('par.y=', par.y, ',child1.y=', child1.y, ',child2.y=', child2.y)

print('-------------------------------------')
Children1.x = 2
child1.y = 2
print('Parent.x=', Parent.x, ',Children1.x=', Children1.x, ',Children2.x=', Children2.x)
print('par.y=', par.y, ',child1.y=', child1.y, ',child2.y=', child2.y)
child1.tet()


In [ ]:
filepath = "I:/陈宁/对账单/2/5.jpg"
iproc_obj = imgproc.ImageProc(filepath)

plt.gcf().set_size_inches(50.0,60.0)
plt.subplot(2,1,1),plt.imshow(iproc_obj.input_img,'gray')
plt.show()

In [1]:
def get_file_content(filePath):
    with open(filePath, 'rb') as fp:
        return fp.read()

image = get_file_content("I:/陈宁/对账单/9股票/9 股票 2 -金_页面_1.jpg")

""" 调用表格文字识别接口 """
#result = client.form(image)
result = client.tableRecognitionAsync(image)
print(result)

NameError: name 'client' is not defined

In [ ]:



""" 如果有可选参数 """
options = {}
options["result_type"] = "excel"
result_id = result['result'][0]['request_id']

""" 带参数调用表格识别结果 """
client.getTableRecognitionResult(result_id, options)

In [ ]:
def get_file_content(filePath):
    with open(filePath, 'rb') as fp:
        return fp.read()

options = {}
options["language_type"] = "CHN_ENG"

image = get_file_content('testimage/441130.jpg')
""" 带参数调用通用文字识别（含位置信息版）, 图片参数为本地图片 """
resutl=client.basicGeneral(image, options)
print(resutl)

In [ ]:
import cv2
options = {}
options["language_type"] = "CHN_ENG"
image = cv2.imread('testimage/608107.jpg')

""" 带参数调用通用文字识别, 图片参数为本地图片 """
resutl=client.image_to_string(image, options)
print(resutl)